# Feature Engineering for Malawi Flood Prediction

In [44]:
# % pip install pyshp
# % pip install geopandas

In [45]:
import pandas as pd
import numpy as np
import geopandas as gpd
from tqdm import tqdm
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Reading the data
*shapes* is a open-source dataset containing polygons of administrative regions in Malawi

*water* is a open-source dataset containing polygons of water bodies in the Africa continent

In this way, we can categorise the features we generate into 2 types: Square features and Region features

In [47]:
# read in data files
train = pd.read_csv("/content/drive/MyDrive/Final Project/Train.csv")
shapes = gpd.read_file("/content/drive/MyDrive/Final Project/mwi_adm_nso_20181016_shp/mwi_admbnda_adm3_nso_20181016.shp")
water = gpd.read_file('/content/drive/MyDrive/Final Project/africawaterbody/Africa_waterbody.shp')

## Helper functions

We define a *haversine* function to calculate the angular distance between two sets of coordinates. 

The *encodePolygon* function is to match each Square to an administrative region provided in *shapes*

In [48]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371
    return c * r

In [49]:
# function to match Square_ID to administrative region provided in shape file 
def encodePolygon(points, polygons):
    numPoints = len(points)
    numPolygons = len(polygons)
    for idx, point in tqdm(enumerate(points)):
        row = np.zeros(numPolygons)
        for idx2, polygon in enumerate(polygons):
            if polygon.contains(point):
                row[idx2] = 1
                break
        if idx == 0:
            matrix = row
        else:
            matrix = np.vstack((matrix,row))
    return matrix.transpose()    

## Data preprocessing

In [50]:
train2015 = train.drop([col for col in train.columns if '2019' in col],axis=1)
precip = [i for i in range(1,18)]
train2015.columns = ['X','Y','target','elevation']+precip+['LC_Type1_mode','id']
train2015['sum'] = train2015[precip].apply(lambda x: x.sum(),axis=1)
train2015['mean'] = train2015[precip].apply(lambda x: x.mean(),axis=1)

## Obtaining region features

Region features refer to features related to each administrative region, for instance, average rainfall in the region and average elevation in the region

In [51]:
# match Square_ID to administrative region and calculate flooding extent, total rainfall and elevation for each region
points = []
for point in zip(train2015.X.values,train2015.Y.values):
    points.append(Point(point[0],point[1]))
polygons = shapes.geometry.values
    
point_polygon_matrix = encodePolygon(points,polygons)
numPoints = point_polygon_matrix.sum(axis=1).transpose()
shapes = shapes.assign(num_points = numPoints)

# rainfall
rainfall = train2015['sum'].values
rainfall_polygons = point_polygon_matrix.dot(rainfall.transpose())
rainfall_polygons = rainfall_polygons / numPoints
# elevation
elevation = train2015.elevation.values
elevation_polygons = point_polygon_matrix.dot(elevation.transpose())
elevation_polygons = elevation_polygons / numPoints

shapes = shapes.assign(rainfall=rainfall_polygons)
shapes = shapes.assign(elevation=elevation_polygons)


16466it [02:26, 112.18it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


In [52]:
point_polygon_matrix.shape

(368, 16466)

In [53]:
shapes = shapes[shapes.num_points!=0]
shapes = shapes.reset_index()
shapes.columns = ['poly_idx' if x=='index' else x for x in shapes.columns]

In [54]:
shapes.columns = ['poly_rainfall' if x=='rainfall' else x for x in shapes.columns]
shapes.columns = ['poly_elevation' if x=='elevation' else x for x in shapes.columns]


In [55]:
# join with shapes -- get polygon features
a,b = np.where(point_polygon_matrix==1) #coordinates of matrix
polygon_map = pd.DataFrame({'poly_idx':a,'square_idx':b})
features = train2015.reset_index()

#square_idx for mapping polygon_map and features
features.columns = ['square_idx' if x=='index' else x for x in features.columns]
features = features.merge(polygon_map,on='square_idx',how='outer')
features.poly_idx = features.poly_idx.fillna(-1)

#poly_idx for mapping features with shapes
features = features.merge(shapes[['poly_idx','poly_rainfall','poly_elevation']],on='poly_idx',how='left')
features.loc[features.poly_rainfall.isna(),'poly_rainfall'] = features['sum']
features.loc[features.poly_elevation.isna(),'poly_elevation'] = features['elevation']

In [56]:
# get diff between shape and polygon
features['elev_diff'] = features['elevation'] - features['poly_elevation']
features['rainfall_diff'] = features['sum'] - features['poly_rainfall']

## Generate features from water body dataset

Using the water body dataset, we can generate an additional Region feature -- number of water bodies in each Region. We can also generate an additional Square feature -- closest water body to each Square. This is useful because from the EDA we see that Squares close to water bodies are most affected by the floods.

In [57]:
# get water features (number of water bodies in each region, distance to closest water body)
water = water.reset_index()
water.columns = ['water_idx' if x=='index' else x for x in water.columns]
water['centroid_lng'] = water.geometry.centroid.x
water['centroid_lat'] = water.geometry.centroid.y
water_centroids = []
for poly in water.geometry.values:
    water_centroids.append(poly.centroid)

water_point_polygon_matrix = encodePolygon(water_centroids,polygons)

a,b = np.where(water_point_polygon_matrix==1)
water_polygon_map = pd.DataFrame({'poly_idx':a,'water_idx':b})
# get number of water bodies in each polygon
water_polygon_count = water_polygon_map.groupby('poly_idx').count().reset_index()
water_polygon_count.columns = ['poly_idx','water_count']
features = features.merge(water_polygon_count,on='poly_idx',how='left')
features.water_count = features.water_count.fillna(0)
# get distance to closest water body
water_polygon_map = water_polygon_map.merge(water[['water_idx','centroid_lng','centroid_lat']],on='water_idx')
closest_water= features.merge(water_polygon_map,on='poly_idx',how='left')
closest_water= closest_water.dropna()
closest_water= closest_water[['square_idx','X','Y','centroid_lng','centroid_lat']]
closest_water['water_dist'] = closest_water[['X','Y','centroid_lng','centroid_lat']].apply(lambda x: haversine(x.X,x.Y,x.centroid_lng,x.centroid_lat),axis=1)
closest_water = closest_water.groupby('square_idx')['water_dist'].min().to_frame().reset_index()
features = features.merge(closest_water,on='square_idx',how='left')
features.water_dist = features.water_dist.fillna(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """
833it [00:02, 385.30it/s]


In [58]:
feature_cols = ['sum','mean','elevation','poly_rainfall','poly_elevation',
                'elev_diff','rainfall_diff','water_count','water_dist']

## Save feature file

In [60]:
features.to_pickle('features.pkl')